data for spy option:

In [ ]:
import os
import pandas as pd
import yfinance as yf

# ============================================================
# CONFIGURATION
# ============================================================

TICKER = "SPY"

# Where to save the Excel file
OUTPUT_PATH = r"the file path for option"

# If you want ALL expirations, set MAX_EXPIRATIONS = None
# If you only want the first N expirations (e.g. nearest 5), set an integer
MAX_EXPIRATIONS = None  # or e.g. 5


# ============================================================
# FUNCTION: download option chain
# ============================================================

def download_option_chain_for_ticker(ticker: str, max_exps=None) -> pd.DataFrame:
    """
    Download option chains from Yahoo Finance using yfinance for the given ticker.
    Returns a single DataFrame with calls + puts for multiple expirations.
    """
    print(f"Downloading option chain metadata for {ticker}...")
    yf_ticker = yf.Ticker(ticker)

    expirations = yf_ticker.options
    if not expirations:
        raise ValueError(f"No option expirations found for {ticker}.")

    print(f"Found {len(expirations)} expiration dates.")

    if max_exps is not None:
        expirations = expirations[:max_exps]
        print(f"Using only first {len(expirations)} expirations: {expirations}")

    all_rows = []

    for exp in expirations:
        print(f"Downloading option chain for expiration {exp}...")
        chain = yf_ticker.option_chain(exp)

        calls = chain.calls.copy()
        puts = chain.puts.copy()

        # Add common metadata
        calls["type"] = "call"
        puts["type"] = "put"
        calls["expiration"] = exp
        puts["expiration"] = exp

        all_rows.append(calls)
        all_rows.append(puts)

    print("Concatenating all expirations...")
    df_all = pd.concat(all_rows, ignore_index=True)

    # Reorder columns a bit
    cols_order = [
        "type", "expiration", "contractSymbol", "lastTradeDate",
        "strike", "lastPrice", "bid", "ask",
        "change", "percentChange", "volume", "openInterest",
        "impliedVolatility", "inTheMoney", "contractSize", "currency"
    ]
    existing_cols = [c for c in cols_order if c in df_all.columns]
    df_all = df_all[existing_cols]

    # Convert to datetime
    if "lastTradeDate" in df_all.columns:
        df_all["lastTradeDate"] = pd.to_datetime(df_all["lastTradeDate"], errors="coerce")
    df_all["expiration"] = pd.to_datetime(df_all["expiration"], errors="coerce")

    # ⚠ IMPORTANT: drop timezones so Excel is happy
    for col in df_all.columns:
        if pd.api.types.is_datetime64tz_dtype(df_all[col]):
            df_all[col] = df_all[col].dt.tz_convert(None)

    return df_all


# ============================================================
# MAIN: download + save to Excel
# ============================================================

def main():
    print(f"Starting options download for {TICKER}...")
    df_options = download_option_chain_for_ticker(TICKER, max_exps=MAX_EXPIRATIONS)

    print(f"Downloaded {len(df_options)} rows of options data.")

    # Ensure output folder exists
    out_dir = os.path.dirname(OUTPUT_PATH)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)

    print(f"Saving to Excel: {OUTPUT_PATH}")
    with pd.ExcelWriter(OUTPUT_PATH, engine="openpyxl") as writer:
        df_options.to_excel(writer, sheet_name="SPY_Options", index=False)

    print("Done!")


if __name__ == "__main__":
    main()


excel for spy stock price is in another file, since its 

code for model: 

In [ ]:


"""
Backtesting 4 models on SPY using Excel data, with Heston regime mixture
calibrated (partially) from SPY option-implied volatilities.

Models:
  2-0: data -> HMM (fit once, frozen)      -> DKL vs empirical
  2-1: data -> HMM (refit up to t)         -> DKL vs empirical
  2-2: data -> local-vol-like Gaussian     -> DKL vs empirical
  2-3: data -> HMM, Heston, Heston-mixture (with IV-based theta) -> pick min DKL model

Files:

  Price file (daily SPY):
    C:\\Users\\brian\\OneDrive\\桌面\\algorithmic trading\\data.xlsx
    Columns (case-insensitive): date, adj close, ...

  Options file (SPY options from Yahoo Finance script):
    C:\\Users\\brian\\OneDrive\\桌面\\algorithmic trading\\options_data.xlsx
    Sheet: SPY_Options
    Columns include: type, expiration, strike, lastPrice, bid, ask,
                     impliedVolatility, lastTradeDate, etc.
"""

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass


# ============================================================
# 1. Load SPY daily prices (underlying + benchmark)
# ============================================================

def load_excel_prices_spy(filepath: str):
    """
    Load your SPY Excel file.

    Expected columns: 'date', 'adj close' (case-insensitive).
    We treat 'adj close' as both:
      - underlying asset price
      - SPY benchmark price.
    """
    df = pd.read_excel(filepath)

    # Normalize date column
    date_col = None
    for c in df.columns:
        if "date" in c.lower():
            date_col = c
            break
    if date_col is None:
        raise ValueError("No date column found (expected something like 'date').")

    df = df.rename(columns={date_col: "Date"})

    # Find adj close column
    adj_col = None
    for c in df.columns:
        lc = c.lower()
        if "adj" in lc and "close" in lc:
            adj_col = c
            break
    if adj_col is None:
        raise ValueError("No 'adj close' column found.")

    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.sort_values("Date").set_index("Date")

    # Use adj close as underlying and SPY benchmark
    df = df[[adj_col]].copy()
    df.columns = ["underlying"]
    df["spy"] = df["underlying"]

    df = df.dropna()
    return df


# ============================================================
# 2. Load SPY options (from options_data.xlsx)
# ============================================================

def load_spy_options(filepath: str, sheet_name: str = "SPY_Options") -> pd.DataFrame:
    """
    Load SPY options data produced by the Yahoo Finance downloader script.

    Expected columns (subset):
      type, expiration, contractSymbol, lastTradeDate, strike,
      lastPrice, bid, ask, impliedVolatility, volume, openInterest, ...
    """
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Options file not found: {filepath}")

    df_opt = pd.read_excel(filepath, sheet_name=sheet_name)

    # Parse datetimes
    if "expiration" in df_opt.columns:
        df_opt["expiration"] = pd.to_datetime(df_opt["expiration"], errors="coerce")
    if "lastTradeDate" in df_opt.columns:
        df_opt["lastTradeDate"] = pd.to_datetime(df_opt["lastTradeDate"], errors="coerce")

    # Clean impliedVolatility: it is in decimal (e.g. 0.15 for 15%)
    if "impliedVolatility" not in df_opt.columns:
        raise ValueError("Options file must have 'impliedVolatility' column.")

    df_opt = df_opt.dropna(subset=["expiration", "impliedVolatility"])
    return df_opt


# ============================================================
# 3. Basic utilities
# ============================================================

def log_returns(series: pd.Series) -> pd.Series:
    return np.log(series / series.shift(1)).dropna()


def rolling_empirical_distribution(
    returns: pd.Series,
    t_index: int,
    H: int,
    lookback: int,
    n_bins: int = 30
):
    """
    Build empirical distribution of H-day log-returns at time t_index
    using a rolling window of length `lookback`.
    """
    start = max(0, t_index - lookback - H)
    end = t_index - 1
    if end - start <= H:
        return None, None

    H_returns = []
    for k in range(start, end - H + 2):
        block = returns.iloc[k:k+H]
        H_returns.append(block.sum())
    H_returns = np.array(H_returns)

    if len(H_returns) < 10:
        return None, None

    hist, edges = np.histogram(H_returns, bins=n_bins, density=False)
    hist = hist.astype(float)
    if hist.sum() == 0:
        return None, None
    p_emp = hist / hist.sum()
    bin_centers = 0.5 * (edges[:-1] + edges[1:])
    return bin_centers, p_emp


def discrete_kl(p, q, eps=1e-12):
    """
    KL(p||q) for discrete distributions p, q given as arrays on same bins.
    """
    p = np.asarray(p, dtype=float)
    q = np.asarray(q, dtype=float)
    p = np.clip(p, eps, 1.0)
    q = np.clip(q, eps, 1.0)
    p /= p.sum()
    q /= q.sum()
    return np.sum(p * np.log(p / q))


# ============================================================
# 4. Gaussian HMM (univariate returns)
# ============================================================

@dataclass
class GaussianHMM:
    n_states: int = 2
    tol: float = 1e-5
    max_iter: int = 200
    random_state: int = 42
    verbose: bool = False

    def __post_init__(self):
        self.rng = np.random.default_rng(self.random_state)
        self.pi_ = None
        self.A_ = None
        self.mu_ = None
        self.sigma2_ = None

    @staticmethod
    def _logsumexp(x, axis=None):
        m = np.max(x, axis=axis, keepdims=True)
        s = np.log(np.sum(np.exp(x - m), axis=axis, keepdims=True)) + m
        return np.squeeze(s, axis=axis)

    @staticmethod
    def _lognorm_pdf(x, mu, sigma2):
        return -0.5 * np.log(2 * np.pi * sigma2) - 0.5 * (x - mu) ** 2 / sigma2

    def _init_params(self, x):
        K = self.n_states
        T = len(x)
        self.pi_ = np.ones(K) / K
        A = np.eye(K) * 0.9
        A[A == 0] = 0.1 / (K - 1)
        self.A_ = A

        # simple init with k-means-like assignment
        q = self.rng.integers(0, K, size=T)
        for _ in range(3):
            mus = np.array([x[q == k].mean() if np.any(q == k) else x.mean() for k in range(K)])
            q = np.argmin(np.abs(x[:, None] - mus[None, :]), axis=1)
        self.mu_ = np.array([x[q == k].mean() if np.any(q == k) else x.mean() for k in range(K)])
        self.sigma2_ = np.array([x[q == k].var() if np.any(q == k) else x.var() for k in range(K)])
        self.sigma2_[self.sigma2_ <= 1e-10] = 1e-10

    def _emission_logprob(self, x):
        T = len(x)
        logB = np.zeros((T, self.n_states))
        for k in range(self.n_states):
            logB[:, k] = self._lognorm_pdf(x, self.mu_[k], self.sigma2_[k])
        return logB

    def _forward(self, logB):
        T, K = logB.shape
        logA = np.log(self.A_)
        logpi = np.log(self.pi_)
        log_alpha = np.zeros((T, K))
        c = np.zeros(T)

        log_alpha[0] = logpi + logB[0]
        c[0] = self._logsumexp(log_alpha[0])
        log_alpha[0] -= c[0]
        for t in range(1, T):
            for j in range(K):
                log_alpha[t, j] = self._logsumexp(log_alpha[t-1] + logA[:, j]) + logB[t, j]
            c[t] = self._logsumexp(log_alpha[t])
            log_alpha[t] -= c[t]
        loglik = np.sum(c)
        return log_alpha, c, loglik

    def _backward(self, logB, c):
        T, K = logB.shape
        logA = np.log(self.A_)
        log_beta = np.zeros((T, K))
        log_beta[-1] = -c[-1]
        for t in range(T-2, -1, -1):
            for i in range(K):
                log_beta[t, i] = self._logsumexp(logA[i] + logB[t+1] + log_beta[t+1])
            log_beta[t] -= c[t]
        return log_beta

    def _e_step(self, x):
        logB = self._emission_logprob(x)
        log_alpha, c, loglik = self._forward(logB)
        log_beta = self._backward(logB, c)
        log_gamma = log_alpha + log_beta
        log_gamma -= self._logsumexp(log_gamma, axis=1)[:, None]
        gamma = np.exp(log_gamma)

        T, K = gamma.shape
        logA = np.log(self.A_)
        log_xi = np.zeros((T-1, K, K))
        for t in range(T-1):
            M = (
                log_alpha[t][:, None]
                + logA
                + logB[t+1][None, :]
                + log_beta[t+1][None, :]
            )
            M -= self._logsumexp(M)
            log_xi[t] = M
        xi = np.exp(log_xi)
        return gamma, xi, loglik

    def _m_step(self, x, gamma, xi):
        T, K = gamma.shape
        self.pi_ = gamma[0] / gamma[0].sum()
        xi_sum = xi.sum(axis=0)
        gamma_sum = gamma[:-1].sum(axis=0)
        self.A_ = xi_sum / np.maximum(gamma_sum[:, None], 1e-16)
        self.A_ = np.clip(self.A_, 1e-12, None)
        self.A_ /= self.A_.sum(axis=1, keepdims=True)

        for k in range(K):
            w = gamma[:, k]
            wsum = w.sum()
            if wsum <= 1e-16:
                continue
            mu = np.sum(w * x) / wsum
            var = np.sum(w * (x - mu) ** 2) / wsum
            self.mu_[k] = mu
            self.sigma2_[k] = max(var, 1e-12)

    def fit(self, x):
        x = np.asarray(x, dtype=float)
        if self.pi_ is None:
            self._init_params(x)
        prev = -np.inf
        for it in range(self.max_iter):
            gamma, xi, loglik = self._e_step(x)
            self._m_step(x, gamma, xi)
            if self.verbose:
                print(f"EM iter {it+1:3d}: loglik={loglik:.6f}")
            if abs(loglik - prev) < self.tol:
                break
            prev = loglik
        return self

    def predict_proba(self, x):
        x = np.asarray(x, dtype=float)
        logB = self._emission_logprob(x)
        log_alpha, c, _ = self._forward(logB)
        log_beta = self._backward(logB, c)
        log_gamma = log_alpha + log_beta
        log_gamma -= self._logsumexp(log_gamma, axis=1)[:, None]
        return np.exp(log_gamma)

    def next_horizon_distribution(self, x, H=5, n_sims=300, n_bins=30):
        """
        Monte Carlo distribution of H-day returns from fitted HMM.
        """
        x = np.asarray(x, dtype=float)
        gamma = self.predict_proba(x)
        p_ST = gamma[-1]  # last-day posteriors

        K = self.n_states
        H_returns = []
        for _ in range(n_sims):
            # sample next state from p_ST @ A
            p_next = p_ST @ self.A_
            state = np.argmax(np.random.multinomial(1, p_next))
            total = 0.0
            for _h in range(H):
                r = np.random.normal(self.mu_[state], np.sqrt(self.sigma2_[state]))
                total += r
                state = np.argmax(np.random.multinomial(1, self.A_[state]))
            H_returns.append(total)

        H_returns = np.array(H_returns)
        hist, edges = np.histogram(H_returns, bins=n_bins, density=False)
        hist = hist.astype(float)
        if hist.sum() == 0:
            return None, None
        q = hist / hist.sum()
        centers = 0.5 * (edges[:-1] + edges[1:])
        return centers, q


# ============================================================
# 5. Simple "local volatility" model
# ============================================================

def local_vol_distribution(
    returns: pd.Series,
    t_index: int,
    H: int,
    lookback: int,
    n_sims: int = 300,
    n_bins: int = 30
):
    """
    Simple local-vol style model:
      - Estimate mean and std of H-day returns in rolling lookback window.
      - Assume Normal with those parameters and generate samples.
      - Histogram -> model-implied distribution.
    """
    start = max(0, t_index - lookback - H)
    end = t_index - 1
    if end - start <= H:
        return None, None

    H_rets = []
    for k in range(start, end - H + 2):
        H_rets.append(returns.iloc[k:k+H].sum())
    H_rets = np.array(H_rets)
    if len(H_rets) < 10:
        return None, None

    mu = H_rets.mean()
    sigma = H_rets.std(ddof=1)
    if sigma <= 1e-8:
        sigma = 1e-8

    sims = np.random.normal(mu, sigma, size=n_sims)
    hist, edges = np.histogram(sims, bins=n_bins, density=False)
    hist = hist.astype(float)
    if hist.sum() == 0:
        return None, None
    q = hist / hist.sum()
    centers = 0.5 * (edges[:-1] + edges[1:])
    return centers, q


# ============================================================
# 6. Heston simulation + IV-based calibration
# ============================================================

def simulate_heston_H_returns(
    H: int,
    n_sims: int,
    mu: float,
    kappa: float,
    theta: float,
    xi: float,
    rho: float,
    v0: float,
    dt: float = 1.0
):
    """
    Simple Euler Heston simulation for H-day log-returns.
    We simulate S_t with S_0 = 1 and output log(S_H).
    """
    H_returns = []
    for _ in range(n_sims):
        S = 1.0
        v = max(v0, 1e-8)
        for _h in range(H):
            z1 = np.random.normal()
            z2 = np.random.normal()
            # Correlated Brownian motions
            z2_corr = rho * z1 + np.sqrt(max(1.0 - rho**2, 0.0)) * z2
            dW1 = np.sqrt(dt) * z1
            dW2 = np.sqrt(dt) * z2_corr

            # variance process
            v = v + kappa * (theta - v) * dt + xi * np.sqrt(max(v, 0.0)) * dW2
            v = max(v, 1e-10)

            # price process
            S = S * np.exp((mu - 0.5 * v) * dt + np.sqrt(v) * dW1)

        H_returns.append(np.log(S))
    return np.array(H_returns)


def calibrate_theta_from_options(
    df_opt: pd.DataFrame,
    as_of_date: pd.Timestamp,
    min_days: int = 20,
    max_days: int = 90
) -> float:
    """
    Very simple IV-based calibration:

      1. Compute days_to_exp = expiration - as_of_date.
      2. Select options with min_days <= days_to_exp <= max_days.
      3. Compute average implied volatility sigma_IV.
      4. Set theta = sigma_IV^2 (annual variance level).

    If no suitable options found, returns None.
    """
    # Ensure date is naive
    as_of_date = pd.to_datetime(as_of_date).normalize()

    df = df_opt.copy()
    df["days_to_exp"] = (df["expiration"].dt.normalize() - as_of_date).dt.days

    mask = (df["days_to_exp"] >= min_days) & (df["days_to_exp"] <= max_days)
    df_slice = df.loc[mask]

    if len(df_slice) == 0:
        return None

    # Filter out insane IVs
    iv = df_slice["impliedVolatility"].astype(float)
    iv = iv[(iv > 0.0) & (iv < 5.0)]  # 0% < IV < 500%
    if len(iv) == 0:
        return None

    sigma_iv = iv.mean()  # annualized volatility (per year)
    theta = sigma_iv ** 2
    return float(theta)


def heston_regime_mixture_distribution(
    hmm: GaussianHMM,
    x_hist: np.ndarray,
    H: int,
    kappa: float,
    theta: float,
    xi: float,
    rho: float,
    r_annual: float = 0.02,
    n_sims: int = 300,
    n_bins: int = 30
):
    """
    Build mixture-of-Heston-regimes distribution:

      q_mix(x) = sum_k p_k q_Heston^{(k)}(x),

    where p_k are HMM posterior probabilities at current time,
    and each regime k has its own initial variance v0_k derived
    from the HMM state variance (v0_k ~ 252 * sigma2_k).

    Heston parameters (kappa, theta, xi, rho) are provided by
    an IV-based calibration from options.
    """
    # HMM posteriors for history
    gamma = hmm.predict_proba(x_hist)
    p_regime = gamma[-1]  # shape (K,)
    K = hmm.n_states

    # Map from HMM variance to Heston initial variance per regime
    v0_list = 252.0 * hmm.sigma2_.copy()
    v0_list[v0_list <= 1e-10] = 1e-10

    mu = r_annual / 252.0  # daily drift

    # Simulate regime-specific Heston distributions
    samples_per_regime = []
    for k in range(K):
        H_rets_k = simulate_heston_H_returns(
            H=H,
            n_sims=n_sims,
            mu=mu,
            kappa=kappa,
            theta=theta,
            xi=xi,
            rho=rho,
            v0=v0_list[k],
            dt=1.0
        )
        samples_per_regime.append(H_rets_k)

    # Build common histogram edges
    all_samps = np.concatenate(samples_per_regime)
    low = np.percentile(all_samps, 0.5)
    high = np.percentile(all_samps, 99.5)
    if low == high:
        low -= 1e-4
        high += 1e-4
    edges = np.linspace(low, high, n_bins + 1)
    centers = 0.5 * (edges[:-1] + edges[1:])

    # Hist per regime
    q_k = []
    for k in range(K):
        hist_k, _ = np.histogram(samples_per_regime[k], bins=edges, density=False)
        hist_k = hist_k.astype(float)
        if hist_k.sum() == 0:
            hist_k = np.ones_like(hist_k, dtype=float)
        q_k.append(hist_k / hist_k.sum())
    q_k = np.stack(q_k, axis=0)  # shape (K, n_bins)

    # Mixture using HMM posterior weights p_regime
    q_mix = np.zeros_like(q_k[0])
    for k in range(K):
        q_mix += p_regime[k] * q_k[k]
    q_mix = np.clip(q_mix, 1e-12, None)
    q_mix /= q_mix.sum()
    return centers, q_mix


# ============================================================
# 7. Backtesting framework
# ============================================================

def align_and_kl(bin_emp, p_emp, bin_model, q_model):
    """
    Interpolate model distribution q_model onto empirical bin grid
    and compute KL(p_emp || q_model_interp).
    """
    if bin_emp is None or p_emp is None or bin_model is None or q_model is None:
        return None
    q_interp = np.interp(bin_emp, bin_model, q_model, left=1e-12, right=1e-12)
    q_interp = np.clip(q_interp, 1e-12, None)
    q_interp = q_interp / q_interp.sum()
    return discrete_kl(p_emp, q_interp)


def backtest_model(
    df: pd.DataFrame,
    df_opt: pd.DataFrame,
    model_mode: str,
    H: int = 5,
    lookback: int = 252,
    n_bins: int = 30,
    n_sims: int = 300,
    hmm_states: int = 2,
    r_annual: float = 0.02
):
    """
    Backtest a model mode: '2-0', '2-1', '2-2', '2-3'.

    Trading rule (simple):
      - Compute model-implied expected H-day return E[R] from distribution.
      - If E[R] > 0 and KL(p_emp || q_model) <= tau: go long underlying for 1 day.
      - Else: stay flat.
    """
    r_u = log_returns(df["underlying"])
    r_spy = log_returns(df["spy"])

    # align length
    r_spy = r_spy.reindex(r_u.index).dropna()
    r_u = r_u.reindex(r_spy.index)

    idx = r_u.index
    n = len(r_u)

    # initial HMM for mode 2-0
    if model_mode == "2-0":
        hmm = GaussianHMM(n_states=hmm_states, verbose=False)
        train_end = min(lookback, n - H - 1)
        hmm.fit(r_u.iloc[:train_end].values)

    equity = [1.0]
    spy_equity = [1.0]
    actions = []

    for t_index in range(lookback, n - H):
        # progress print every 50 steps
        if (t_index - lookback) % 50 == 0:
            print(f"{model_mode}: step {t_index - lookback} / {n - lookback - H}")

        # empirical distribution at time t_index
        bin_emp, p_emp = rolling_empirical_distribution(
            r_u, t_index, H=H, lookback=lookback, n_bins=n_bins
        )

        if bin_emp is None:
            actions.append(0)
            next_u = equity[-1] * np.exp(0.0)
            next_spy = spy_equity[-1] * np.exp(r_spy.iloc[t_index])
            equity.append(next_u)
            spy_equity.append(next_spy)
            continue

        r_next = r_u.iloc[t_index]
        current_date = idx[t_index]

        q_model = None
        bin_model = None

        # Model 2-0: single HMM fit, frozen
        if model_mode == "2-0":
            bin_model, q_model = hmm.next_horizon_distribution(
                r_u.iloc[:t_index].values, H=H, n_sims=n_sims, n_bins=n_bins
            )

        # Model 2-1: re-fit HMM each step on all past data
        elif model_mode == "2-1":
            hmm_step = GaussianHMM(n_states=hmm_states, verbose=False)
            hmm_step.fit(r_u.iloc[:t_index].values)
            bin_model, q_model = hmm_step.next_horizon_distribution(
                r_u.iloc[:t_index].values, H=H, n_sims=n_sims, n_bins=n_bins
            )

        # Model 2-2: local-vol style model on H-day returns
        elif model_mode == "2-2":
            bin_model, q_model = local_vol_distribution(
                r_u, t_index, H=H, lookback=lookback, n_sims=n_sims, n_bins=n_bins
            )

        # Model 2-3: HMM, Heston (IV-calibrated), and Heston-mixture -> choose by min KL
        elif model_mode == "2-3":
            # HMM distribution
            hmm_step = GaussianHMM(n_states=hmm_states, verbose=False)
            hmm_step.fit(r_u.iloc[:t_index].values)
            x_hist = r_u.iloc[:t_index].values
            bin_hmm, q_hmm = hmm_step.next_horizon_distribution(
                x_hist, H=H, n_sims=n_sims, n_bins=n_bins
            )

            # Single local-vol approximation
            bin_heston_local, q_heston_local = local_vol_distribution(
                r_u, t_index, H=H, lookback=lookback, n_sims=n_sims, n_bins=n_bins
            )

            # Calibrate theta from options for this date (rough, uses near-term maturity)
            theta_est = calibrate_theta_from_options(df_opt, current_date, min_days=20, max_days=90)
            if theta_est is None:
                # fallback: use HMM-derived variance
                theta_est = float(252.0 * np.mean(hmm_step.sigma2_))

            # Heston hyperparameters (can be tuned)
            kappa = 1.5
            xi = 0.5
            rho = -0.5

            # Heston regime mixture
            bin_mix, q_mix = heston_regime_mixture_distribution(
                hmm_step,
                x_hist,
                H=H,
                kappa=kappa,
                theta=theta_est,
                xi=xi,
                rho=rho,
                r_annual=r_annual,
                n_sims=n_sims,
                n_bins=n_bins
            )

            # KLs on empirical bins
            kl_hmm = align_and_kl(bin_emp, p_emp, bin_hmm, q_hmm)
            kl_heston_local = align_and_kl(bin_emp, p_emp, bin_heston_local, q_heston_local)
            kl_mix = align_and_kl(bin_emp, p_emp, bin_mix, q_mix)

            # Choose the model with smallest KL
            kl_list = []
            model_list = []

            if kl_hmm is not None:
                kl_list.append(kl_hmm)
                model_list.append(("HMM", bin_hmm, q_hmm))
            if kl_heston_local is not None:
                kl_list.append(kl_heston_local)
                model_list.append(("HESTON_LOCAL", bin_heston_local, q_heston_local))
            if kl_mix is not None:
                kl_list.append(kl_mix)
                model_list.append(("HESTON_MIX", bin_mix, q_mix))

            if len(kl_list) == 0:
                bin_model, q_model = None, None
            else:
                best_idx = int(np.argmin(kl_list))
                _, bin_model, q_model = model_list[best_idx]

        else:
            raise ValueError(f"Unknown model_mode: {model_mode}")

        kl_val = align_and_kl(bin_emp, p_emp, bin_model, q_model)

        # expected H-day return from q_model
        if (bin_model is None) or (q_model is None):
            exp_H = 0.0
        else:
            exp_H = np.sum(bin_model * q_model)

        tau = 0.2  # KL threshold; tune as needed
        if (kl_val is not None) and (kl_val <= tau) and (exp_H > 0.0):
            position = 1.0
        else:
            position = 0.0

        actions.append(position)

        new_equity = equity[-1] * np.exp(position * r_next)
        equity.append(new_equity)

        new_spy = spy_equity[-1] * np.exp(r_spy.iloc[t_index])
        spy_equity.append(new_spy)

    # stop at n - H (exclusive) to match equity length
    res_index = idx[lookback:n - H]
    equity_series = pd.Series(equity[1:], index=res_index, name=f"model_{model_mode}")
    spy_series = pd.Series(spy_equity[1:], index=res_index, name="SPY_benchmark")
    actions_series = pd.Series(actions, index=res_index, name=f"action_{model_mode}")
    return equity_series, spy_series, actions_series


# ============================================================
# 8. Run all 4 models
# ============================================================

def run_all_models(price_path: str, options_path: str):
    df = load_excel_prices_spy(price_path)
    df_opt = load_spy_options(options_path, sheet_name="SPY_Options")

    H = 5
    lookback = 252  # 1 year for empirical estimation

    equity_dict = {}
    spy_ref = None

    for mode in ["2-0", "2-1", "2-2", "2-3"]:
        print(f"Starting backtest for model {mode}...")
        eq, spy, act = backtest_model(
            df,
            df_opt,
            model_mode=mode,
            H=H,
            lookback=lookback,
            n_bins=30,
            n_sims=300,   # <= here: 300 sims per step
            hmm_states=2,
            r_annual=0.02
        )
        equity_dict[mode] = eq
        if spy_ref is None:
            spy_ref = spy
        print(f"Finished backtest for model {mode}.")

    all_df = pd.concat(
        [equity_dict["2-0"], equity_dict["2-1"],
         equity_dict["2-2"], equity_dict["2-3"], spy_ref],
        axis=1
    )
    all_df.columns = ["model_2_0", "model_2_1", "model_2_2", "model_2_3", "SPY"]
    return all_df


def plot_reward_vs_time(all_df: pd.DataFrame):
    """
    Make 3 plots:
      - last quarter (approx 63 days)
      - last year   (approx 252 days)
      - last 5 years (approx 1260 days)
    """
    all_df = all_df.dropna()

    def plot_window(ax, df_window, title):
        ax.plot(df_window.index, df_window["model_2_0"], label="Model 2-0 (HMM once)")
        ax.plot(df_window.index, df_window["model_2_1"], label="Model 2-1 (HMM rolling)")
        ax.plot(df_window.index, df_window["model_2_2"], label="Model 2-2 (Local vol)")
        ax.plot(df_window.index, df_window["model_2_3"], label="Model 2-3 (Hybrid HMM-Heston)")
        ax.plot(df_window.index, df_window["SPY"],       label="SPY benchmark", linestyle="--")
        ax.set_title(title)
        ax.set_xlabel("Date")
        ax.set_ylabel("Equity (normalized)")
        ax.grid(True)

    n = len(all_df)
    quarter = 63
    year = 252
    five_years = 1260

    fig, axes = plt.subplots(3, 1, figsize=(10, 12), sharex=False)

    # 1 quarter
    if n > quarter:
        df_q = all_df.iloc[-quarter:]
        plot_window(axes[0], df_q, "Reward vs Time (Last Quarter)")

    # 1 year
    if n > year:
        df_y = all_df.iloc[-year:]
        plot_window(axes[1], df_y, "Reward vs Time (Last Year)")

    # 5 years
    if n > five_years:
        df_5y = all_df.iloc[-five_years:]
        plot_window(axes[2], df_5y, "Reward vs Time (Last 5 Years)")

    axes[0].legend(loc="best")
    plt.tight_layout()
    plt.show()


if __name__ == "__main__":
    price_path = r"the file path for stock price"
    options_path = r"the file path for option"

    all_results = run_all_models(price_path, options_path)
    plot_reward_vs_time(all_results)
